##DATA CLEANING NOTEBOOK FOR GROUP A

This notebook contains the necessary jupiter notebook codes needed to clean the dirty customer support ticket dataset for our ML Internship week 1 project.

It basically does the following:
1. Load the necessary libraries
2. Loads and reads the dataset in a csv format
3. Performs basic EDA to check for missing data
Displays the top 10 and last 10 rows of the data to inspect the text
4. Performs the needed cleaning to correct labels, remove unwanted characters, emojis, duplicates, unwanted standalone texts/numbers and deal with missing data.
5. Saves the clean dataset to (customer_support_tickets_clean_500.csv)



>## 1.  Install libraries



In [29]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import httpx

In [30]:
import sys
print('sys.executable:', sys.executable)
print('python version:', sys.version)
import pandas as pd, matplotlib, sklearn
print('pandas', pd.__version__)
print('matplotlib', matplotlib.__version__)
print('sklearn', sklearn.__version__)

sys.executable: c:\Users\ankom\AppData\Local\Programs\Python\Python313\python.exe
python version: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
pandas 2.3.3
matplotlib 3.10.7
sklearn 1.7.2


## 2. Load data

In [31]:
DATA_PATH = r'C:\Users\ankom\Thrive_Internship_ML_A\customer_support_tickets_dirty.csv'

In [32]:
df = pd.read_csv(DATA_PATH)
print("✅ Data loaded. Shape:", df.shape)
df.head()

✅ Data loaded. Shape: (520, 4)


,text,label,ticket_id,agent_name
0,Where can I find API rate limits documentation?,Othr,1000,Dana
1,Can I migrate data from Freshdesk?,Othr,1001,Bob
2,Cannot update billing address; page keeps relo...,billing,1002,Charlie
3,Looking for a product roadmap or upcoming feat...,other,1003,Dana
4,Dark mode toggled but UI stays light.,Tech-support,1004,Alice


## Read Data

In [33]:
df = pd.read_csv(DATA_PATH)

print("✅ Loaded dataset.")
print("Shape:", df.shape)


✅ Loaded dataset.
Shape: (520, 4)


## Exploratory Data Analysis (EDA)

In [34]:
#df.info()
#df.isnull().sum()


In [35]:
print("Columns:", df.columns.tolist())
print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum())

Columns: ['text', 'label', 'ticket_id', 'agent_name']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        495 non-null    object
 1   label       495 non-null    object
 2   ticket_id   520 non-null    int64 
 3   agent_name  520 non-null    object
dtypes: int64(1), object(3)
memory usage: 16.4+ KB
None

Missing values per column:
text          25
label         25
ticket_id      0
agent_name     0
dtype: int64


Finding Percentage of missing data

In [36]:
missing_percent = df.isnull().mean() * 100
print(missing_percent)


text          4.807692
label         4.807692
ticket_id     0.000000
agent_name    0.000000
dtype: float64


We considered replacing the missing data with "unknown" because the percentage of missing values is less than 5%

**EDA- First looking at the Data**

In [17]:
# diplaying first 10 rows
df.head(10)


,text,label,ticket_id,agent_name
0,Where can I find API rate limits documentation?,Othr,1000,Dana
1,Can I migrate data from Freshdesk?,Othr,1001,Bob
2,Cannot update billing address; page keeps relo...,billing,1002,Charlie
3,Looking for a product roadmap or upcoming feat...,other,1003,Dana
4,Dark mode toggled but UI stays light.,Tech-support,1004,Alice
5,Promo code SPRING15 not applied at checkout.,billing,1005,Alice
6,Incorrect role assigned; need admin access for...,account,1006,Dana
7,File export to CSV generates corrupted file.,technical,1007,Dana
8,Overcharged by GHS 120 on my last bill. 😡,billing,1008,Dana
9,NaN,account,1009,Dana


In [37]:
# display last 10 rows
df.tail(10)

,text,label,ticket_id,agent_name
510,Requesting invoice to be sent monthly instead ...,Othr,1510,Bob
511,Dark mode toggled but UI stays light. 😡,Tech,1511,Dana
512,Do you offer discounts for students or NGOs?,Other,1512,Charlie
513,Live chat widget not loading on homepage. 😡,TECHNICAL,1513,Alice
514,Refund not received after 7 days. 😡,Billing,1514,Alice
515,Currency converted incorrectly; expected USD. ...,Billng,1515,Charlie
516,Can't verify phone number; code not sent.,ACCOUNT,1516,Alice
517,Dark mode toggled but UI stays light.,Tech,1517,Charlie
518,Search results page is blank after entering a ...,Tech,1518,Dana
519,Payment failed during checkout with card endin...,Billng,1519,Alice


# Checking the unique values of label to help detect any error

In [ ]:
df.label.unique()

array(['Othr', 'billing', 'other', 'Tech-support', 'account', 'technical',
       'Accnt', 'OTHER', 'TECHNICAL', 'ACCOUNT', 'BILLING', 'Other ', nan,
       'Billng', 'Billing ', ' Account ', ' Tech'], dtype=object)

Our dataset is messy due to missing, inconsistent labels, unwanted information, funny characters and emojis . Cleaning it ensures that the final data is complete, consistent, and ready for accurate analysis or modeling.

## Data cleaning

In [39]:
import pandas as pd
import re
import os

#  Configuring the path
# Use the DATA_PATH variable defined in the previous cells
file_path = DATA_PATH


#  Loading the data
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find the CSV. Tried: {file_path}. Update file_path and try again.")

print(f"Loaded {file_path} — rows: {len(df)}, columns: {list(df.columns)}")

# Function to clean text (preserve case)
def clean_text_preserve_case(text):
    """
    Cleans text by:
    - Removing standalone '12345' and standalone 'br' words
    - Preserving case
    - Keeping digits, spaces, ., ?, @, $
    - Removing other unwanted special characters
    - Normalizing extra spaces
    """
    if not isinstance(text, str):
        return text

    # Remove standalone '12345'
    text = re.sub(r'(?<!\d)12345(?!\d)', '', text)

    # Remove standalone 'br' (case-insensitive)
    text = re.sub(r'\bbr\b', '', text, flags=re.IGNORECASE)

    # Remove unwanted characters but KEEP @ and $
    text = re.sub(r'[^A-Za-z0-9\s\.\?\@\$\-]', '', text)

    # Remove spaces before punctuation
    text = re.sub(r'\s+([.?])', r'\1', text)

    # Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Label corrections
label_corrections = {
    'othr': 'Other',
    'tech-support': 'Technical',
    'technical-support': 'Technical',
     'TECHNICAL': 'Technical',
    'billng': 'Billing',
    'BILLING': 'Billing',
    'accnt': 'Account',
    'account': 'Account',
    'Account': 'Account',
    'other': 'Other',
    'tech': 'Technical'
}

def correct_labels(text):
    """Apply label corrections case-insensitively."""
    if not isinstance(text, str):
        return text
    cleaned = text.strip()
    for wrong, right in label_corrections.items():
        cleaned = re.sub(rf'\b{wrong}\b', right, cleaned, flags=re.IGNORECASE)
    return cleaned

#  Identify text columns
text_columns = [c for c in ['text', 'label', 'agent_name'] if c in df.columns]
if not text_columns:
    raise ValueError("None of the expected text columns ('text','label','agent_name') are present.")

#  Cleaning pipeline
df = df.drop_duplicates(keep='first')
df = df.fillna("unknown")

# Apply cleaning to text columns
for col in text_columns:
    df[col] = df[col].astype(str).apply(clean_text_preserve_case)

# Apply corrections to the label column specifically
if 'label' in df.columns:
    df['label'] = df['label'].astype(str).apply(correct_labels)

#  Diagnostics
if 'text' in df.columns:
    after_count_12345 = df['text'].astype(str).str.contains(r'\b12345\b', regex=True, na=False).sum()
    after_count_br = df['text'].astype(str).str.contains(r'\bbr\b', regex=True, na=False).sum()
    print(f"✅ '12345' occurrences removed: {after_count_12345 == 0}")
    print(f"✅ 'br' occurrences removed: {after_count_br == 0}")

#  Save cleaned file
out_path = "customer_support_tickets_clean_500.csv"
df.to_csv(out_path, index=False)
print(f"\n✅ Saved cleaned CSV to: {out_path}")
print(df.head(15).to_string(index=False))

Loaded C:\Users\ankom\Thrive_Internship_ML_A\customer_support_tickets_dirty.csv — rows: 520, columns: ['text', 'label', 'ticket_id', 'agent_name']
✅ '12345' occurrences removed: True
✅ 'br' occurrences removed: True

✅ Saved cleaned CSV to: customer_support_tickets_clean_500.csv
                                                     text     label  ticket_id agent_name
          Where can I find API rate limits documentation?     Other       1000       Dana
                       Can I migrate data from Freshdesk?     Other       1001        Bob
      Cannot update billing address page keeps reloading.   Billing       1002    Charlie
      Looking for a product roadmap or upcoming features.     Other       1003       Dana
                    Dark mode toggled but UI stays light. Technical       1004      Alice
             Promo code SPRING15 not applied at checkout.   Billing       1005      Alice
   Incorrect role assigned need admin access for my team.   Account       1006       Dana


In [40]:
# display last 10 rows
df.head(10)

,text,label,ticket_id,agent_name
0,Where can I find API rate limits documentation?,Other,1000,Dana
1,Can I migrate data from Freshdesk?,Other,1001,Bob
2,Cannot update billing address page keeps reloa...,Billing,1002,Charlie
3,Looking for a product roadmap or upcoming feat...,Other,1003,Dana
4,Dark mode toggled but UI stays light.,Technical,1004,Alice
5,Promo code SPRING15 not applied at checkout.,Billing,1005,Alice
6,Incorrect role assigned need admin access for ...,Account,1006,Dana
7,File export to CSV generates corrupted file.,Technical,1007,Dana
8,Overcharged by GHS 120 on my last bill.,Billing,1008,Dana
9,unknown,Account,1009,Dana


Now, we will recheck how clean our first 10 and last 10 row look.

In [41]:
# display last 10 rows
df.tail(10)

,text,label,ticket_id,agent_name
510,Requesting invoice to be sent monthly instead ...,Other,1510,Bob
511,Dark mode toggled but UI stays light.,Technical,1511,Dana
512,Do you offer discounts for students or NGOs?,Other,1512,Charlie
513,Live chat widget not loading on homepage.,Technical,1513,Alice
514,Refund not received after 7 days.,Billing,1514,Alice
515,Currency converted incorrectly expected USD.,Billing,1515,Charlie
516,Cant verify phone number code not sent.,Account,1516,Alice
517,Dark mode toggled but UI stays light.,Technical,1517,Charlie
518,Search results page is blank after entering a ...,Technical,1518,Dana
519,Payment failed during checkout with card endin...,Billing,1519,Alice


Finally, the next line of code rechecks for missing data.

In [42]:
# Rechecking for missing data

print("Columns:", df.columns.tolist())
print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum())

Columns: ['text', 'label', 'ticket_id', 'agent_name']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        520 non-null    object
 1   label       520 non-null    object
 2   ticket_id   520 non-null    int64 
 3   agent_name  520 non-null    object
dtypes: int64(1), object(3)
memory usage: 16.4+ KB
None

Missing values per column:
text          0
label         0
ticket_id     0
agent_name    0
dtype: int64


So dataset is soo clean.